# Module 1 - Key Sustainability Initiatives (KSIs) FROM Companies' Annual reports

 1. Extract top global Companies list, from [Report evaluated by PwC with Bloomberg](https://github.com/prak112/esg-profile/blob/4bebecd15332ae550aca1627835b451ce2bdab4f/dataset) and build a dataframe
 <br></br>
 2. Web Search for each Company's Annual Report/Annual Sustainability Report
    * **If available, Extract KSIs**
    * If NA, mark companies KSI data as NA or null **(for probable interpretation as - ESG proxy data/outsourced ESG activities)**


In [2]:
# Import neccessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Extraction libraries

! pip install PyPDF2 --quiet
print("Installed PyPDF2")
import PyPDF2
import re

In [6]:
# Extract PDF data

pdf_file = open(r"\dataset\global-top-100-companies-june-2020-update.pdf", 'rb')
pdf_data = PyPDF2.PdfFileReader(pdf_file)
print("Total Pages of Report:", pdf_data.numPages)

Total Pages of Report: 17


In [8]:
# Extract tables from pdf_data
pages = [10, 11, 12, 13, 14]
table_data = []

for page in pages:
    table = pdf_data.getPage(page).extractText()
    table_data.append(table)

print(table_data[:1])

['PwC \n| Global Top 100 companies by market capitalisation\n11Top 100 global companies 1\n-20\nSource: Bloomberg with PwC analysis\n30 June 2020\n31 March 2020\n31 Dec 2019\nRank\nCompany name\nLocation\nSector\nRank +/\n-(VS Mar 2020)\nRank +/\n-(VS Dec 2019)\nMarket  \ncapitalisation \n\n($bn)\nRank\nMarket \n\ncapitalisation \n\n($bn)\nRank\nMarket \ncapitalisation \n($bn)\n1SAUDI ARABIAN OI \nSaudi Arabia \nOil & Gas \n001,741 \n1 1,602 \n1 1,879 \n2APPLE INC \nUnited States \nTechnology \n101,568 \n3 1,113 \n2 1,305 \n3MICROSOFT CORP \nUnited States \nTechnology \n-101,505 \n2 1,200 \n3 1,203 \n4AMAZON.COM INC \nUnited States \nConsumer Services \n011,337 \n4 971 5 916 5ALPHABET INC\n-A United States \nTechnology \n0-1953 5 799 4 923 6FACEBOOK INC\n-A United States \nTechnology \n10629 7 475 6 585 7TENCENT\nMainland China \nTechnology \n12599 8 469 9 461 8ALIBABA GRP\n-ADR \nMainland China \nConsumer Services \n-2-1577 6 522 7 569 9BERKSHIRE HATH\n-A United States \nFinancials \n

In [10]:
#' N/A \n' ---'space CAPSletter character CAPSletter space nextline'-->to omit

ranks = list(range(1, 101))
company_data = []

for table in list(range(0,5)):
    regex = re.findall('[A-Z\s\W?a-z]+(?:\s)+(?:\n)+', table_data[table]) # regex and filter_data avoid all formatting-space. 
    filter_data = [ele for ele in regex if ele != ' ' and ' \n' and ',' and '-']
    if table == 0:
        company_data.append(filter_data[2:])
    else:
        company_data.append(filter_data[4:])

In [12]:
company_data[2]

['EXXON MOBIL CORP \nUnited States \nOil & Gas \n',
 'PING AN \nMainland China \nFinancials \n',
 'PEPSICO INC \nUnited States \nConsumer Goods \n',
 'PFIZER INC \nUnited States \nHealth Care \n',
 "L'OREAL \nFrance \nConsumer Goods \n",
 'COMCAST CORP\n-A United States \nConsumer Services \n',
 'ABBVIE INC \nUnited States \nHealth Care \n',
 'SAP SE Germany \nTechnology \n',
 'ORACLE CORP \nUnited States \nTechnology \n',
 'SALESFORCE.COM \nUnited States \nTechnology \n',
 'CHEVRON CORP \nUnited States \nOil & Gas \n',
 'ABC-H Mainland China \nFinancials \n',
 'ABBOTT LABS \nUnited States \nHealth Care \n',
 'ELI LILLY & CO \nUnited States \nHealth Care \n',
 'ASML HOLDING NV \nNetherlands \nTechnology \n',
 'NIKE INC \n-CL B \nUnited States \nConsumer Goods \n',
 'NOVO NORDISK\n-B Denmark \nHealth Care \n',
 'PROSUS NV \nNetherlands \nTechnology \n',
 'RELIANCE INDS \nIndia \nOil & Gas \n',
 'THERMO FISHER \nUnited States \nHealth Care \n']

In [14]:
top1to20 = company_data[0]
top20to40 = company_data[1]
top40to60 = company_data[2]
top60to80 = company_data[3]
top80to100 = company_data[4]

-----------------
-----------------

### [*PROGRESS So Far:*]
* > *data_dict* output clear with **top1to20, top60to80 & top80to100** , i.e.,

    > *All elements are equally distributed amongst columns :  True*
* > ERROR in **top20to40 & top40t060** : 
    * > Exchange _'CCB-H Mainland China'_ in *Company_Name* to:
        * 'CCB-H' in *Company_Name*
        * 'China' in *Location*
    * > Change _'NIKE INC \n-CL B \nUnited States'_ to _'NIKE INC-CL B \nUnited States'_
* > ERROR in **top60to80** :
    * > Company Class (*-A, -B, -H..*) to be filtered out/combined with *Company_Name*

--------------
--------------

In [23]:
?str.find

Docstring:
S.find(sub[, start[, end]]) -> int

Return the lowest index in S where substring sub is found,
such that sub is contained within S[start:end].  Optional
arguments start and end are interpreted as in slice notation.

Return -1 on failure.
Type:      method_descriptor


In [16]:
data_dict = {}

data = top40to60
company_name = []
location = []
sector = []

for ele in data:
    subs = re.sub('N/A', '', ele).strip()   # substitute N/A with blanks  
    subs = re.sub('\n-REG', '-REG', ele).strip()    # substitute \n-A with -A\n,  substitute \n-REG with -REG\n  
    splits = subs.split('\n')    # split into groups by \n
    stripped = [split.strip() for split in splits if split != 'N/A' and '']     # split & filter elements
    
    try:    # to prevent any indexing errors
        # Collect Company_Name as a list --> Add as Value to 'Company_Name' Key in data_dict
        if stripped[0].find('Mainland China') == -1:
            company_name.append(stripped[0])
        else:
            stripped[0].replace('Mainland China','China')
            company_name.append(stripped[0])
        
        # Collect Location as a list --> Add as Value to 'Location' Key in data_dict
        if stripped[1] != '-A Mainland China':
            location.append(stripped[1])
        else:
            location.append(stripped[1].replace('-A Mainland China','China'))

        # Collect Sector as a list --> Add as Value to 'Sector' Key in data_dict
        sector.append(stripped[2])

    except IndexError:
        continue


In [18]:
data_dict['Rank'] = ranks[40:60]
data_dict['Company_Name'] = company_name
data_dict['Location'] = location
data_dict['Sector'] = sector

#stripped
for key, val in data_dict.items():
    print(key ,':', val,'\n')

Rank : [41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60] 

Company_Name : ['EXXON MOBIL CORP', 'PING AN', 'PEPSICO INC', 'PFIZER INC', "L'OREAL", 'COMCAST CORP', 'ABBVIE INC', 'SAP SE Germany', 'ORACLE CORP', 'SALESFORCE.COM', 'CHEVRON CORP', 'ABC-H Mainland China', 'ABBOTT LABS', 'ELI LILLY & CO', 'ASML HOLDING NV', 'NIKE INC', 'NOVO NORDISK', 'PROSUS NV', 'RELIANCE INDS', 'THERMO FISHER'] 

Location : ['United States', 'Mainland China', 'United States', 'United States', 'France', '-A United States', 'United States', 'Technology', 'United States', 'United States', 'United States', 'Financials', 'United States', 'United States', 'Netherlands', '-CL B', '-B Denmark', 'Netherlands', 'India', 'United States'] 

Sector : ['Oil & Gas', 'Financials', 'Consumer Goods', 'Health Care', 'Consumer Goods', 'Consumer Services', 'Health Care', 'Technology', 'Technology', 'Oil & Gas', 'Health Care', 'Health Care', 'Technology', 'United States', 'Health Care', 'Technolog

In [20]:
print("All elements are equally distributed amongst columns : ", len(ranks[20:40])==len(company_name)==len(location)==len(sector))
print(len(ranks[20:40]))
print(len(company_name))
print(len(location))
print(len(sector))


All elements are equally distributed amongst columns :  False
20
20
20
18


In [22]:

def trim_data(company_data):
    """
    substitute tags, strip spaces & split string into elements
    
    Args:
    company_data - list containing Company_Name, Location, Sector
    
    Returns:
    new_data - dictionary containing formatted data as per columns

    """
    new_data = {}
    columns = ['Company_Name', 'Location', 'Sector']

    for data in company_data:
        subs = re.sub('N/A', '', data).strip() # substitute N/A with blanks
        subs = re.sub('\n-', '-', data).strip() # substitute \n- with -
        splits = subs.split('\n') # split into elements by \n
        stripped = [split.strip() for split in splits if split != ''] 
        for column, element in zip(columns, stripped):  # iterate columns and data to update dictionary
            new_data[column] = element # ERROR-cause: repetition of column name, instead add all values relevant to column in one-go.

    return new_data